# Spectra geographically bins


In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd

# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
# import geopandas as gpd
# from shapely.geometry import Polygon

%matplotlib inline
from matplotlib import pyplot as plt

import drifters.utils as ut
import pynsitu as pin
from GDP_lib import root_dir

In [2]:
from dask.distributed import Client

if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    # cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    # cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=3, processes=3, walltime="04:00:00")
    w = cluster.scale(jobs=5)
    # from dask_jobqueue import PBSCluster
    # cluster = PBSCluster()
    # w = cluster.scale(jobs=8) # 5 for gps, 8 for argos
else:
    from dask.distributed import LocalCluster

    cluster = LocalCluster()

client = Client(cluster)
client

/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:237: FutureWarning: extra has been renamed to worker_extra_args. You are still using it (even if only set to []; please also check config files). If you did not set worker_extra_args yet, extra will be respected for now, but it will be removed in a future release. If you already set worker_extra_args, extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.min

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.65:8787/status,
Dashboard: http://10.148.1.65:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.65:37519,Workers: 0
Dashboard: http://10.148.1.65:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Choose key 

In [3]:
key = "gps"  # 'gps' or 'argos'
key2 = ""  # or '_corrected'

# Load

In [4]:
# LOAD files with velocities and accelerations computed

parquet = os.path.join(root_dir, key + "_av_time" + key2 + ".parquet")
# parquetd = os.path.join(root_dir, key + "_av_time_w.parquet")

# drop all variables but relevant ones
col = [
    "id",
    "time",
    "lon",
    "lat",
    "x",
    "y",
    "ve",
    "vn",
    "ae",
    "an",
    "vex",
    "vny",
    "aex",
    "any",
    "vex_diff",
    "vny_diff",
]
df = dd.read_parquet(parquet).reset_index()[col].persist()
# dfd = dd.read_parquet(parquetd).reset_index()[col].persist()

In [5]:
df

,id,time,lon,lat,x,y,ve,vn,ae,an,vex,vny,aex,any,vex_diff,vny_diff
npartitions=178,,,,,,,,,,,,,,,,
,int64,datetime64[ns],float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
len(df)

126753036

In [7]:
df.nunique().compute()


KeyboardInterrupt



In [ ]:
nb_traj = df.id.nunique().compute()

_________________
# Generate Spectra

In [6]:
df = df.rename(columns={"time": "date"})
df["ven2"] = df.ve**2 + df.vn**2

# add time in hours
time_unit = pd.Timedelta("1H")
t_ref = pd.Timestamp(2000, 1, 1)
df["time"] = (df["date"] - t_ref) / time_unit

In [7]:
T_str = "60D"  # days
dt = "1H"  # sampling

N = int(pd.Timedelta(T_str) / pd.Timedelta(dt))  # output size
T = pd.Timedelta(T_str) / time_unit  # must be in the same units than time

columns = [
    "lon",
    "lat",
]
columns0 = columns + ["ve", "vn"]
columns1 = columns + ["ae", "an"]
columns2 = columns + ["vex", "vny"]
columns3 = columns + ["aex", "any"]
columns4 = columns + ["vex_diff", "vny_diff"]
columns5 = columns + ["x", "y"]

labels = ["ven", "aen", "vxy", "axy", "vxydiff", "xy"]
Columns = {
    "ven": columns0,
    "aen": columns1,
    "vxy": columns2,
    "axy": columns3,
    "vxydiff": columns4,
    "xy": columns5,
}

In [8]:
def process_uv(lon, lat, u, v, N, dt, **kwargs):
    """Wraps spectral calculation: add complex velocity
    Assumes the time series is regularly sampled

    Parameters:
    -----------
        u, v: pd.Series
            zonal, meridional index by time (in days)
        N: int,
            length of the spectrum
        dt: float
            Time sampling in days
        **kwargs:
            passed to mit_equinox.drifters.get_spectrum
    """
    if lon is None:
        uv = None
    else:
        uv = u + 1j * v
    return pin.tseries.get_spectrum(uv, N, dt=dt, **kwargs)

In [9]:
df.head()

,id,date,lon,lat,x,y,ve,vn,ae,an,vex,vny,aex,any,vex_diff,vny_diff,ven2,time
0,22192,2001-07-01 02:00:00,-94.795609,5.53791,0.000000,0.000000,0.6460,-0.2900,NaN,NaN,0.640596,-0.277060,NaN,NaN,0.640586,-0.277083,0.501416,13130.0
1,22192,2001-07-01 03:00:00,-94.774597,5.52852,2328.165360,-1038.334652,0.6466,-0.2901,-2.777825e-07,-3.472219e-07,0.649771,-0.294109,1.698938e-06,-3.157259e-06,0.649776,-0.294098,0.502250,13131.0
2,22192,2001-07-01 04:00:00,-94.753387,5.51876,4678.387112,-2117.504284,0.6440,-0.2925,-1.361113e-06,-1.180553e-06,0.656827,-0.308859,2.221023e-06,-5.036993e-06,0.656843,-0.308824,0.500292,13132.0
3,22192,2001-07-01 05:00:00,-94.731918,5.50841,7057.436407,-3261.867565,0.6368,-0.2986,-2.000001e-06,-1.694444e-06,0.660713,-0.317772,-6.206238e-08,8.546555e-08,0.660741,-0.317713,0.494676,13133.0
4,22192,2001-07-01 06:00:00,-94.710457,5.49807,9435.722447,-4405.037660,0.6296,-0.3047,-2.000001e-06,-1.694444e-06,0.660724,-0.317611,6.839744e-08,4.095455e-09,0.660764,-0.317528,0.489238,13134.0


In [10]:
len(df)

41635507

### Compute spectra

In [11]:
# pin.drifters.
group = tuple(df.get_partition(0)["id"].loc[0].values.compute())[0]
dfg = df.groupby("id").get_group(group).compute()

out = pin.drifters.time_window_processing(
    dfg, process_uv, columns0, T, N, id_label="id", dt=dt, geo=True
)
# dfg.set_index("time").plot(x="lon", y="lat")


Df_chunked = {}
for l in Columns:
    df_chunked = (
        df.groupby("id")
        .apply(
            pin.drifters.time_window_processing,
            process_uv,
            Columns[l],
            T,
            N,
            id_label="id",
            dt=dt,
            geo=True,
            meta=out,
        )
        .persist()
    )
    # recompute date
    df_chunked["date"] = t_ref + df_chunked.index * time_unit
    # rename x/y
    df_chunked = df_chunked.rename(columns=dict(x="lon", y="lat"))
    # add nrj
    # df_chunked['ven2'] = df.ve**2 +df.vn**2

    Df_chunked[l] = df_chunked

In [9]:
Df_chunked["xy"].tail()

,lon,lat,id,0.0,0.016666666666666666,0.03333333333333333,0.05,0.06666666666666667,0.08333333333333333,0.1,...,-0.15,-0.13333333333333333,-0.11666666666666667,-0.1,-0.08333333333333333,-0.06666666666666667,-0.05,-0.03333333333333333,-0.016666666666666666,date
140436.0,-172.495639,-11.871011,133214.0,3.006250e+15,7.396051e+14,2.626960e+10,5.949926e+09,1.747403e+09,1.110348e+09,4.272085e+08,...,3.521635e+08,7.888988e+08,5.445478e+08,2.600478e+07,3.599553e+09,2.047074e+10,2.472590e+10,3.986670e+10,7.708600e+14,2016-01-08 12:00:00
141156.0,-173.397731,-11.855721,133214.0,3.055312e+15,7.789024e+14,5.653823e+10,1.186186e+10,1.159896e+09,2.008727e+08,6.438361e+07,...,9.212385e+06,2.353636e+08,3.865734e+08,1.883257e+08,1.640823e+09,5.738156e+09,3.813265e+09,4.051052e+10,7.523443e+14,2016-02-07 12:00:00
141876.0,-173.772518,-12.217947,133214.0,3.096833e+15,7.767399e+14,2.585844e+10,4.147512e+09,4.862059e+08,3.054070e+08,5.698088e+08,...,1.958103e+07,1.612496e+08,1.128564e+08,1.577030e+08,1.066023e+09,1.646711e+09,1.646830e+09,3.785751e+09,7.765987e+14,2016-03-08 12:00:00
142596.0,-173.096795,-13.234011,133214.0,3.042310e+15,7.922205e+14,2.148700e+10,2.410912e+10,4.198264e+09,1.066765e+09,4.489320e+08,...,1.454164e+08,3.080449e+08,2.294268e+09,7.354775e+09,1.963089e+09,9.077579e+09,8.240415e+09,4.192332e+10,7.302662e+14,2016-04-07 12:00:00
143316.0,-172.332552,-14.438307,133214.0,2.978883e+15,7.439642e+14,3.947125e+09,2.999322e+09,7.597460e+08,2.325743e+09,1.430164e+09,...,1.860726e+08,5.086584e+08,3.781564e+09,4.398189e+09,4.885281e+09,1.462578e+10,4.343257e+10,4.153438e+10,7.430707e+14,2016-05-07 12:00:00


In [ ]:
did = Df_chunked["aen"].id.compute()

In [ ]:
print(f"{did.nunique()} trajectories with more than T = 60 days among the {nb_traj}")

In [ ]:
len(did)

## Store spectra

In [12]:
DF = Df_chunked
DFs = {}
for l in DF:
    DFs[l] = DF[l].rename(
        columns={i: str(i) for i in list(DF[l].columns) if isinstance(i, float)}
    )

In [13]:
root_dir = "/home1/datawork/mdemol/GDP"


def store_spectra(Df, root_dir=root_dir):
    for var_key in Columns:
        parquet = os.path.join(
            root_dir, key + "_" + var_key + "_spectra" + key2 + ".parquet"
        )
        df = Df[var_key].repartition(partition_size="100MB").persist()
        df.to_parquet(parquet, engine="pyarrow", overwrite=True)
        print(var_key)


store_spectra(DFs)

ven
aen
vxy
axy
vxydiff
xy


---

# Geographically binned

https://github.com/apatlpo/mit_equinox/blob/master/parcels/spectra_binned.ipynb

In [14]:
DF = {}
labels = ["ven", "aen", "vxy", "axy", "vxydiff", "xy"]
for l in labels:
    parquet = os.path.join(root_dir, key + "_" + l + "_spectra" + key2 + ".parquet")
    df = dd.read_parquet(parquet)
    df = df.rename(
        columns={
            i: float(i)
            for i in list(df.columns)
            if i not in ["lon", "lat", "id", "date"]
        }
    )
    DF[l] = df

In [15]:
DF["xy"].head()

,lon,lat,id,0.0,0.016666666666666666,0.03333333333333333,0.05,0.06666666666666667,0.08333333333333333,0.1,...,-0.15,-0.13333333333333333,-0.11666666666666667,-0.1,-0.08333333333333333,-0.06666666666666667,-0.05,-0.03333333333333333,-0.016666666666666666,date
114008.0,112.141853,-44.230513,11273030.0,1.844374e+12,5.265224e+11,1.262812e+09,2.001975e+09,1.487455e+08,2.638608e+09,4.927136e+08,...,6.679869e+07,6.721481e+07,2.495761e+08,1.283264e+09,5.521480e+08,3.578865e+09,1.148163e+10,2.082868e+10,4.785267e+11,2013-01-02 08:00:00
114728.0,112.778701,-43.098082,11273030.0,4.599380e+12,1.200654e+12,5.251643e+09,1.672801e+09,2.204176e+08,3.334396e+08,1.046112e+09,...,1.842898e+08,5.492016e+08,6.831924e+08,9.479491e+08,8.170194e+09,1.371005e+10,1.780136e+10,7.778551e+09,1.127874e+12,2013-02-01 08:00:00
115448.0,114.253899,-42.629487,11273030.0,8.859628e+12,1.827954e+12,3.250247e+09,3.461412e+09,4.530008e+07,6.517959e+08,4.616104e+08,...,1.451463e+07,1.310884e+07,1.501080e+08,8.051470e+07,3.830177e+08,8.972052e+08,2.650756e+09,2.512392e+10,3.037203e+12,2013-03-03 08:00:00
116168.0,117.353551,-42.701941,11273030.0,1.672633e+13,3.173580e+12,6.046932e+09,9.104136e+08,4.997942e+09,2.875937e+09,1.614609e+09,...,2.306066e+08,3.513580e+08,2.863105e+08,2.619233e+08,6.388402e+08,1.175635e+09,5.590931e+09,1.050949e+11,6.084225e+12,2013-04-02 08:00:00
116888.0,119.642472,-42.348902,11273030.0,2.981449e+13,8.881094e+12,3.605030e+10,1.573638e+10,6.790941e+09,2.405648e+09,2.159317e+06,...,3.898855e+08,2.314524e+08,2.654523e+07,1.302754e+08,5.590529e+07,3.226603e+08,9.723002e+08,3.274661e+10,7.333116e+12,2013-05-02 08:00:00


In [16]:
# bin geographically
dl = 2
lon_bins = np.arange(
    -180.0, 180.0 + dl, dl
)  # CAUTION : add dl to upper bound (prevent from pb with last binning intervals)
lat_bins = np.arange(-90, 90 + dl, dl)

### Computing

In [17]:
DF_geo = {}
Ds = {}
for l in DF:
    # DF[l].drop(columns=['ven2'], axis=1, inplace=True)
    DF[l]["lon"] = (DF[l]["lon"] + 180) % 360 - 180
    DF[l]["lon_cut"] = DF[l]["lon"].map_partitions(pd.cut, bins=lon_bins)
    DF[l]["lat_cut"] = DF[l]["lat"].map_partitions(pd.cut, bins=lat_bins)
    DF_geo[l] = (
        DF[l].groupby(["lon_cut", "lat_cut"]).mean().compute()
    )  # MEAN SPECTRUM OVER ALL SEGMENT IN THE LON, LAT bins

    # Converting pandas dataframe in xarray dataset:
    index = pd.MultiIndex.from_arrays(
        [
            DF_geo[l].index.map(lambda v: v[0].mid),
            DF_geo[l].index.map(lambda v: v[1].mid),
        ],
        names=("lon_cut", "lat_cut"),
    )
    ds = (
        (
            pd.melt(
                DF_geo[l]
                .set_index(index)
                .drop(columns=["id", "lon", "lat"])
                .reset_index(),
                id_vars=[
                    "lon_cut",
                    "lat_cut",
                ],
                var_name="frequency",
            ).rename(
                columns={
                    "lon_cut": "lon_bins",
                    "lat_cut": "lat_bins",
                    "value": "E_" + l,
                }
            )
        )
        .to_xarray()
        .set_index(index=["lon_bins", "lat_bins", "frequency"])
        .unstack()
    )
    ds["frequency"] = ds["frequency"].astype(float)
    ds = ds.sortby("frequency")
    # ds = ds.chunk({'frequency': 100})
    Ds[l] = ds
    print(l)

# nb counts
dsc = (
    DF[l]
    .groupby(["lon_cut", "lat_cut"])
    .size()
    .compute()
    .to_frame("nb_geobins")
    .set_index(index)
    .reset_index()
    .to_xarray()
    .rename({"lon_cut": "lon_bins", "lat_cut": "lat_bins"})
    .set_index(index=["lon_bins", "lat_bins"])
    .unstack()
)

ven
aen
vxy
axy
vxydiff
xy


In [18]:
ds = xr.merge(list(Ds.values()) + [dsc])

# set attrs
ds.E_xy.attrs = {
    "long_name": "Power density spectra X = x + jy",
    "units": r"$m^2/cpd$",
    "description": "LOWESS method",
}
ds.E_ven.attrs = {
    "long_name": "Power density spectra v = ve + jvn",
    "units": r"$m^2/s^2/cpd$",
    "description": "LOWESS method",
}
ds.E_vxy.attrs = {
    "long_name": "Power density spectra v = vx + jvy",
    "units": r"$m^2/s^2/cpd$",
    "description": "geoid method",
}
ds.E_vxydiff.attrs = {
    "long_name": "Power density spectra v = vx + jvy",
    "units": r"$m^2/s^2/cpd$",
    "description": "finite differentiation method",
}
ds.E_axy.attrs = {
    "long_name": "Power density spectra a = ax + jay",
    "units": r"$m^2/s^4/cpd$",
    "description": "from position",
}
ds.E_aen.attrs = {
    "long_name": "Power density spectra a = ae + jan",
    "units": r"$m^2/s^4/cpd$",
    "description": "from ve,vn",
}

ds.frequency.attrs = {"long_name": "frequency", "units": "cpd"}
ds.nb_geobins.attrs = {
    "long_name": "nb_geobins",
    "description": "number of T days time window spectra per bins",
}

In [19]:
ds["E_venw2"] = ds["E_ven"] * (ds["frequency"] * 2 * np.pi / 3600 / 24) ** 2
ds["E_vxyw2"] = ds["E_vxy"] * (ds["frequency"] * 2 * np.pi / 3600 / 24) ** 2

In [20]:
ds.E_venw2.attrs = {
    "long_name": "E_ven $  \omega^2$",
    "units": r"$m^2/s^4/cpd$",
    "description": "from position",
}
ds.E_vxyw2.attrs = {
    "long_name": "E_vxy $  \omega^2$",
    "units": r"$m^2/s^4/cpd$",
    "description": "from ve,vn",
}

In [21]:
ds

<xarray.Dataset>
Dimensions:     (lon_bins: 180, lat_bins: 90, frequency: 1440)
Coordinates:
  * lon_bins    (lon_bins) float64 -179.0 -177.0 -175.0 ... 175.0 177.0 179.0
  * lat_bins    (lat_bins) float64 -89.0 -87.0 -85.0 -83.0 ... 85.0 87.0 89.0
  * frequency   (frequency) float64 -12.0 -11.98 -11.97 ... 11.95 11.97 11.98
Data variables:
    E_ven       (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_aen       (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_vxy       (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_axy       (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_vxydiff   (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_xy        (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    nb_geobins  (lon_bins, lat_bins) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
    E_venw2     (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_vxyw2     (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan

In [20]:
ds.nb_geobins.sum()

<xarray.DataArray 'nb_geobins' ()>
array(159366)


## Store geo spectra

In [29]:
key = "argos"

In [27]:
root_dir = "/home1/datawork/mdemol/GDP"
zarr = os.path.join(root_dir, key + f"_geospectra_{int(dl)}" + key2 + ".zarr")

In [28]:
ds.to_zarr(zarr, mode="w")

In [30]:
ds_reload = xr.open_zarr(zarr).persist()
ds_reload

<xarray.Dataset>
Dimensions:     (lon_bins: 180, lat_bins: 90, frequency: 1440)
Coordinates:
  * frequency   (frequency) float64 -12.0 -11.98 -11.97 ... 11.95 11.97 11.98
  * lat_bins    (lat_bins) float64 -89.0 -87.0 -85.0 -83.0 ... 85.0 87.0 89.0
  * lon_bins    (lon_bins) float64 -179.0 -177.0 -175.0 ... 175.0 177.0 179.0
Data variables:
    E_aen       (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_axy       (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_ven       (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_venw2     (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_vxy       (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_vxydiff   (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_vxyw2     (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_xy        (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    nb_geobins  (lon_bins, lat_bins) int64 dask.array<chunksize=(180, 90), meta=np.ndarray>

In [31]:
cluster.close()